In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

NameError: name 'url' is not defined

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [4]:
sunscreen_url = []
for i in range (1,38):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/sunscreen.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    sunscreen_list = soup.find_all('div', class_='product-item-info')
    for item in sunscreen_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            sunscreen_url.append(link['href'])
print(len(sunscreen_url))

443


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [ ]:
sunscreen_url

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [31]:
sunscreen_url[446]

'https://www.stylevana.com/en_EU/kose-suncut-uv-perfect-milk-super-water-proof-spf50-pa-60g.html'

In [5]:
sunscreens = []

In [6]:
for link in sunscreen_url[0:443]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    sunscreen_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    sunscreens.append(sunscreen_info)

In [7]:
print(len(sunscreens))

443


In [7]:
sunscreens

[{'name': 'Kao - Biore UV Aqua Rich Watery Essence SPF 50+ PA++++ 2019 Edition',
  'details': 'Usage: Day\n\n\nProduct Type: Essence & Serums\n\n\nProduct Unit: 50g\n\n\nSPF: SPF 30 - 50\n\n\nPA: ++++',
  'ingredients': 'Ethylhexyl methoxycinnamate, cross-polymer of (lauryl methacrylate / sodium methacrylate), alkyl benzoate (C12-15), bisethylhexyloxyphenol methoxyphenyl triazine, hexyl diethylaminohydroxybenzoylbenzoate, dimethicone, ethylhexyl triazone , Dimethylsilylated silica, DPG, xylitol, dextrin palmitate, (acrylates / alkyl (C10-30)) crosspolymer, (dimethicone / vinyldimethicone) crosspolymer, polysilicone-9, glyceryl stearate, AMP, Vinyl dimethicone / methicone silsesquioxane) crosspolymer, agar, isoceteth-20, alkyl (C30-45) methicone, polyvinyl alcohol, olefin (C30-45), sodium hydroxide BG, PG, hyaluronic acid Na, royal jelly, orange fruit extract, grapefruit fruit extract, lemon fruit extract, phenoxyethanol, EDTA-2Na, BHT, perfume'},
 {'name': 'Isntree - Hyaluronic Acid Wa

## 3. PASARLO A DATAFRAME

In [8]:
sunscreens_df = pd.DataFrame(sunscreens)
sunscreens_df

,name,details,ingredients,price,best-seller
0,Canmake - Mermaid Skin Gel UV SPF 50+ PA++++ -...,Cruelty Free: Yes\n\n\nProduct Type: Sunscreen...,"Water, Ethylhexyl Methoxycinnamate (Aka Octino...",10.79€,Best Sellers
1,Kao - Biore UV Aqua Rich Watery Essence SPF 50...,Usage: Day\n\n\nProduct Type: Essence & Serums...,"Ethylhexyl methoxycinnamate, cross-polymer of ...",9.09€,Best Sellers
2,COSRX - Aloe Soothing Sun Cream,Cruelty Free: Yes\n\n\nUsage: Day\n\n\nProduct...,"Water, Ethylhexyl Methoxycinnamate, Glycerin, ...",10.19€,Best Sellers
3,Rohto Mentholatum - Skin Aqua Sunscreen Super ...,Product Type: Sunscreen\n\n\nSPF: Above SPF 50...,[],10.89€,Best Sellers
4,ROVECTIN - Skin Essentials Aqua Soothing UV Pr...,"Usage: Day, Night\n\n\nProduct Type: Sunscreen...",[],12.29€,Best Sellers
...,...,...,...,...,...
438,Rohto Mentholatum - Acnes UV Tint Milk Sunscre...,no info,no info,no info,no
439,The Saem - Eco Earth Waterproof Sun Cream - 50g,no info,no info,no info,no
440,SUNTIQUE - I'm Safe for Baby & Silver SPF35 PA...,no info,no info,no info,no
441,SKIN79 - Water Wrapping Moist Cool Sun Stick S...,no info,no info,no info,no


In [10]:
sunscreens = sunscreens_df.to_csv('../data_/sunscreens.csv')